# Assignment 10 scratch space

In [1]:
%%file ../testing_mpi.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
print('size=%d, rank=%d' % (size, rank))

Overwriting ../testing_mpi.py


In [5]:
!mpiexec -n 4 python ../testing_mpi.py

size=4, rank=0
size=4, rank=2
size=4, rank=1
size=4, rank=3


In [12]:
%%file ../mpi2.py
from mpi4py import MPI
rank = MPI.COMM_WORLD.Get_rank()

a = 6.0
b = 3.0
if rank == 0:
        print(a + b)
if rank == 1:
        print(a * b)
if rank == 2:
        print(max(a,b))
        
print("end of MPI" + str(rank))

Overwriting ../mpi2.py


In [13]:
!mpiexec -n 4 python ../mpi2.py

9.0
end of MPI0
18.0
end of MPI1
6.0
end of MPI2
end of MPI3


In [54]:
%%file ../mpi3.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1)
m = [0]
        

if rank == 0:
        print("Process", rank, "before receiving has the number", randNum[0])
        comm.Recv(randNum, source=1)
        print("Process", rank, "received the number", randNum[0])
        randNum *= 2
        comm.Send(randNum*10, dest=1)

if rank == 1:
        randNum = numpy.random.random_sample(1)
        print("Process", rank, "drew the number", randNum[0])
        comm.Send(randNum, dest=0)
        m = numpy.zeros(1)
        comm.Recv(m, source=0)
        print("Process", rank, "now has the number", m)

Overwriting ../mpi3.py


In [55]:
!mpiexec -n 2 python ../mpi3.py

Process 0 before receiving has the number 0.0
Process 1 drew the number 0.981705310531
Process 0 received the number 0.981705310531
Process 1 now has the number [ 19.63410621]


In [86]:
%%file ../mpi3_2.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

m = dict()    
m['init'] = 1
if rank == 0:
        print("Process", rank, "before receiving has the number", m)
        m = comm.recv( source=1)
        print("Process", rank, "received the number", m)
        m['hey'] = 'hi hi hi'
        comm.send(m, dest=1)
        m['just0'] = True
        print("Process", rank, "finally received the number", m)

if rank == 1:
        m['hi from 1'] = 'asdf'
        print("Process", rank, "drew the number", m)
        comm.send(m, dest=0)
        m = comm.recv( source=0)
        m['just1'] = True
        print("Process", rank, "now has the number", m)

Overwriting ../mpi3_2.py


In [87]:
!mpiexec -n 2 python ../mpi3_2.py

Process 0 before receiving has the number {'init': 1}
Process 1 drew the number {'hi from 1': 'asdf', 'init': 1}
Process 0 received the number {'hi from 1': 'asdf', 'init': 1}
Process 0 finally received the number {'hi from 1': 'asdf', 'just0': True, 'hey': 'hi hi hi', 'init': 1}
Process 1 now has the number {'hey': 'hi hi hi', 'just1': True, 'init': 1, 'hi from 1': 'asdf'}


In [82]:
%%file ../mpi6.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1)
diffNum = numpy.random.random_sample(1)

if rank == 1:
        randNum = numpy.random.random_sample(1)
        print("Process", rank, "drew the number", randNum[0])
        comm.Isend(randNum, dest=0) 
        # no need to wait after I send
        req = comm.Irecv(randNum, source=0)
        req.Wait()
        print("Process", rank, "received the number", randNum[0])
        
if rank == 0:
        print("Process", rank, "before receiving has the number", randNum[0])
        req = comm.Irecv(randNum, source=1)
        req.Wait()
        print("Process", rank, "received the number", randNum[0])
        randNum *= 2
        comm.Isend(randNum, dest=1)
        # no need to wait after I send

Writing ../mpi6.py


In [83]:
!mpiexec -n 2 python ../mpi6.py

Process 0 before receiving has the number 0.0
Process 1 drew the number 0.176685198743
Process 0 received the number 0.176685198743
Process 1 received the number 0.353370397487


In [106]:
%%file ../mpi7.py
import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

unique_number = numpy.array(rank)


if rank == 0:
    print("Process", rank, "is alive")

    comm.Barrier()
    print("Process", rank, "has passed Barrier")

    k = 0
    for i in range(1, comm.Get_size()): #must not include self!
        print(i)
        k += comm.recv(source=i)
    
    print("Process", rank, "totalled the number", k)

    # no need to wait after I send
if rank > 0 :
    print("Process", rank, " is alive")
    comm.isend(unique_number, dest=0)
    comm.Barrier()
    print("Process", rank, "has sent number ", unique_number)
    print("Process", rank, "has passed Barrier")

        


Overwriting ../mpi7.py


In [107]:
!mpiexec -n 20 python ../mpi7.py

Process 0 is alive
Process 1  is alive
Process 2  is alive
Process 3  is alive
Process 4  is alive
Process 5  is alive
Process 6  is alive
Process 9  is alive
Process 7  is alive
Process 12  is alive
Process 13  is alive
Process 14  is alive
Process 16  is alive
Process 8  is alive
Process 10  is alive
Process 11  is alive
Process 17  is alive
Process 15  is alive
Process 19  is alive
Process 18  is alive
Process 15 has sent number  15
Process 15 has passed Barrier
Process 16 has sent number  16
Process 16 has passed Barrier
Process 0 has passed Barrier
1
Process 3 has sent number  3
Process 3 has passed Barrier
Process 5 has sent number  5
Process 5 has passed Barrier
Process 12 has sent number  12
Process 12 has passed Barrier
Process 14 has sent number  14
Process 14 has passed Barrier
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Process 0 totalled the number 190
Process 11 has sent number  11
Process 11 has passed Barrier
Process 2 has sent number  2
Process 2 has passed Barrier
P

In [92]:
#Comm.Scatter(self, sendbuf, recvbuf, int root=0)
#recvobj = Comm.scatter(self, sendobj, int root=0)

In [ ]:
# it seems like 0 is hte master node, and all nodes (including 0 are worker nodes...)
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0:
    data = [(i+1)**2 for i in range(size)]
else:
    data = None # worker node
data = comm.scatter(data, root=0)
# so, it does get sent to node 0 as well...which I HATE


assert data == (rank+1)**2

In [118]:
%%file ../mpi_integral.py
import numpy
from math import acos, cos
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

def integral(a_i, h, n):
    integ = 0.0
    for j in range(n):
        a_ij = a_i + (j + 0.5) * h
        integ += cos(a_ij) * h
    return integ

pi = 3.14159265359
a = 0.0
b = pi / 2.0
dest = 0
my_int = numpy.zeros(1)
integral_sum = numpy.zeros(1)

# Initialize value of n only if this is rank 0
if rank == 0:
    n = numpy.full(shape=1, fill_value=400, dtype=int) # default value
else:
    n = numpy.zeros(shape=1, dtype=int)

# Broadcast n to all processes
print("Process ", rank, " before n = ", n[0])
comm.Bcast(n, root=0)
print("Process ", rank, " after n = ", n[0])

# Compute partition
h = (b - a) / (n * size) # calculate h *after* we receive n
a_i = a + rank * h * n
my_int[0] = integral(a_i, h, n[0])

# Send partition back to root process, computing sum across all partitions
print("Process ", rank, " has the partial integral ", my_int[0])
comm.Reduce(my_int, integral_sum, MPI.SUM, dest)

# Only print the result in process 0
if rank == 0:
    print('The Integral Sum =', integral_sum[0])

Overwriting ../mpi_integral.py


In [117]:
!mpiexec -n 10 python ../mpi_integral.py

Process  0  before n =  400
Process  1  before n =  0
Process  2  before n =  0
Process  4  before n =  0
Process  5  before n =  0
Process  0  after n =  400
Process  3  before n =  0
Process  3  after n =  400
Process  6  before n =  0
Process  7  before n =  0
Process  7  after n =  400
Process  8  before n =  0
Process  8  after n =  400
Process  9  before n =  0
Process  9  after n =  400
Process  0  has the partial integral  0.156434466045
Process  1  after n =  400
Process  2  after n =  400
Process  1  has the partial integral  0.152582530315
Process  2  has the partial integral  0.144973506296
Process  4  after n =  400
Process  8  has the partial integral  0.0366318245354
Process  5  after n =  400
Process  5  has the partial integral  0.101910213843
Process  6  after n =  400
Process  6  has the partial integral  0.0819895303402
Process  7  has the partial integral  0.0600499924926
Process  9  has the partial integral  0.012311659484
Process  3  has the partial integral  0.1

http://mpi4py.readthedocs.io/en/latest/tutorial.html

Slightly different syntax.

Can use numpy arrays as a buffer
- faster
- must use capital Send and Recv

Can use generic objects
- slower
- must use lowercase Send and Recv
- m = comm.recv(source=1)

## Question 1
Write an MPI program in which the processes with even rank print “Hello” and the processes with odd rank print “Goodbye”. Print the rank along with the message (for example “Goodbye from process 3”). Hint: remember that although the number of processes is fixed when the program starts, the exact number is not known until the Get_size() method is called.

Make sure you have adequate comments and documentation in the code, and you follow good software engineering practices.

Call the program mpi_assignment_1.py and commit it to the same repository you used for Assignment 3.

In [ ]:
import numpy
from math import acos, cos
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

In [128]:
%%file mpi_assignment_1.py
import numpy as np
from mpi4py import MPI

# learn about COMM environment
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# determine the parity of the rank
odd_rank = (rank % 2) == 1

# print
if odd_rank:
    print("Hello from process ", rank)
else: 
    print("Goodbye from process ", rank)

Overwriting mpi_assignment_1.py


In [129]:
!mpiexec -n 5 python mpi_assignment_1.py

Hello from process  1
Goodbye from process  2
Goodbye from process  4
Hello from process  3
Goodbye from process  0


## Question 2

Write an MPI program that does the following for some arbitrary number of processes:

1. Process 0 reads a value from the user and verifies that it is an integer less than 100.
2. Process 0 sends the value to process 1 which multiplies it by its rank.
3. Process 1 sends the new value to process 2 which multiplies it by its rank.
4. This continues for each process, such that process i sends the value to process i+1 which multiplies it by i+1.
5. The last process sends the value back to process 0, which prints the result.


You can use either blocking or non-blocking operations. Any input read from the user must be validated correctly and exceptions handled. Make sure you have adequate comments and documentation in the code, and you follow good software engineering practices.

Call the program mpi_assignment_2.py and commit it to the same repository you used for Assignment 3.

In [173]:
%%file mpi_assignment_2.py
from mpi4py import MPI
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

if size < 2:
    print("Program was expecting more ranks.  It will do it's best and just return your input.")

if rank == 0:
    
    # get the user input
    def is_integer_less_than_100(x):
        try:
            return float(x) == int(x) and int(x)<100
        except:
            return False

    user_integer = input('Enter an integer less than 100: ')
    while not is_integer_less_than_100(user_integer):
        print("Not a valid integer less than 100")
        user_integer = input('Enter an integer less than 100: ')
    current_integer = int(user_integer)
    
    if size > 1:
        # send it to process 1
        comm.isend(current_integer, dest=1)

        # get the final answer and print it
        current_integer = comm.recv(source=size-1)
    print(current_integer)
    
if rank >= 1:
    current_integer = comm.recv(source=rank-1)
    current_integer *= rank
    if rank == size -1:
        comm.isend(current_integer, dest=0)
    else: 
        comm.isend(current_integer, dest=rank + 1)

Overwriting mpi_assignment_2.py


In [180]:
%%file mpi_assignment_2.py
from numpy import empty
from mpi4py import MPI
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

current_integer = empty(1, dtype=int)

if size < 2:
    print("Program was expecting more ranks.  It will do it's best and just return your input.")

if rank == 0:
    
    # get the user input
    def is_integer_less_than_100(x):
        try:
            return float(x) == int(x) and int(x)<100
        except:
            return False

    user_integer = input('Enter an integer less than 100: ')
    while not is_integer_less_than_100(user_integer):
        print("Not a valid integer less than 100")
        user_integer = input('Enter an integer less than 100: ')
    current_integer[0] = int(user_integer)
    
    if size > 1:
        # send it to process 1
        comm.Send(current_integer, dest=1)

        # get the final answer and print it
        comm.Recv(current_integer, source=size-1)
    print(current_integer[0])
    
if rank >= 1:
    comm.Recv(current_integer, source=rank-1)
    current_integer *= rank
    if rank == size -1:
        comm.Isend(current_integer, dest=0)
    else: 
        comm.Isend(current_integer, dest=rank + 1)

Overwriting mpi_assignment_2.py


In [ ]:
!mpiexec -n 2 python mpi_assignment_2.py

Enter an integer less than 100: 

In [155]:
pwd()

'/Users/akunas/Documents/advpython/assignments/ark520_assignment3'

In [149]:

def is_integer_less_than_100(x):
    try:
        return float(x) == int(x) and int(x)<100
    except:
        return False
    
user_integer = input('Enter an integer less than 100: ')
while not is_integer_less_than_100(user_integer):
    print("Not a valid integer less than 100")
    user_integer = input('Enter an integer less than 100: ')


What is your name? 4


In [150]:
def a(int b):
    return b

SyntaxError: invalid syntax (<ipython-input-150-9e853352b6a8>, line 1)

In [144]:
0==None

False

In [164]:
10*10

100